In [169]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import numpy as np
from sklearn.pipeline import Pipeline

In [170]:
df_pos = pd.read_excel('data/output/clean_handlabeled_data.xlsx')


Raghava

## Classifier

In [171]:
df_pos['LABEL'] = df_pos['LABEL'].map({'NEU':'POS','POS':'POS','NEG':'NEG'})

In [172]:
df_pos['LABEL'] = df_pos['LABEL'].map({'POS':1,'NEG':0})

In [173]:
X = df_pos['COMMENT'].values
y = df_pos['LABEL'].values

In [199]:
count = CountVectorizer(max_features=896)
tfidf = TfidfTransformer(smooth_idf=True,use_idf=False)

#X = tdidf_transformer.fit_transform(count).toarray()

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [210]:

X_train_count = count.fit_transform(X_train)

X_test_count = count.fit_transform(X_test)


X_train_tfidf = tfidf.transform(X_train_count).toarray()
X_test_tfidf = tfidf.transform(X_test_count).toarray()



In [211]:
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(779, 896)
(195, 896)


Pseudo labeling

In [212]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train_tfidf, y_train)


# predict the labels on validation dataset
predictions = lin_clf.predict(X_test_tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions, y_test)*100)

Naive Bayes Accuracy Score ->  54.871794871794876


In [213]:
pseudoY_test = lin_clf.predict(X_test_tfidf)

In [214]:
X_test_tfidf.shape

(195, 896)

In [215]:
X = np.vstack((X_train_tfidf, X_test_tfidf))
Y = np.concatenate((y_train, pseudoY_test), axis=0)



In [216]:
pseudo_model = svm.LinearSVC()
pseudo_model.fit(X, Y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [218]:
Accuracyclf = lin_clf.score(X_train_tfidf, y_train)
print ("Accuracy of the lin_clf model, : ", Accuracyclf*100, "%")

Accuracypseudo = lin_clf.score(X, Y)
print ("Accuracy of the lin_clf model: ", Accuracypseudo*100, "%")

Accuracy of the lin_clf model, :  92.42618741976894 %
Accuracy of the lin_clf model:  93.94250513347022 %
